In [1]:
!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import pickle,os
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    ## creating ride_id column
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df

In [4]:
year = 2022
month = 2
taxi_type = 'yellow'

directory_path = f"output/{taxi_type}"

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

df = read_data(input_file)
output_file = f'{directory_path}/{year:04d}-{month:02d}.parquet'

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### standard deviation of the predicted duration

In [6]:
import numpy as np

# Calculate mean
mean = np.mean(y_pred)

# Calculate squared differences
squared_diff = [(pred - mean) ** 2 for pred in y_pred]

# Calculate variance
variance = np.mean(squared_diff)

# Calculate standard deviation
std_deviation = np.sqrt(variance)

print("Standard Deviation:", std_deviation)

Standard Deviation: 5.28140357655334


### Q2. Preparing the output

In [11]:
df_result = df[['ride_id']]
df_result.loc[:, 'predictions'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

C:\Users\padma\AppData\Local\Temp\ipykernel_13548\688827274.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result.loc[:, 'predictions'] = y_pred


In [12]:
df_result.head(2)

,ride_id,predictions
0,2022/02_0,18.527783
1,2022/02_1,23.065782


### Q3. Convert ipynb file to script using the below command from command line